<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

This notebook is created by Zhuo Chen under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
For questions/comments/improvements, email zhuo.chen@ithaka.org or nathan.kelber@ithaka.org<br />
___

# Pandas 3 

**Description:** This notebook describes how to:
* Build a dataset from Constellate
* Make a dataframe from the dataset
* Summarize data in a dataframe
* Group and aggregate data
* Make pivot tables in Pandas

This is the third notebook in a series on learning to use Pandas. 

**Use Case:** For Learners (Detailed explanation, not ideal for researchers)

**Difficulty:** Intermediate

**Knowledge Required:** 
* [Pandas 1](./pandas-1.ipynb)
* [Pandas 2](./pandas-2.ipynb)
* Python Basics ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:** 
* [Python Intermediate 1](./python-intermediate-1.ipynb)
* [Python Intermediate 2](./python-intermediate-2.ipynb)
* [Python Intermediate 4](./python-intermediate-4.ipynb)

**Completion Time:** 60 minutes

**Data Format:** JSONL 

**Libraries Used:** Pandas

**Research Pipeline:** None
___


# Build a dataset from Constellate

The dataset we are going to use for today's lesson is the documents from JSTOR or Portico with the key word "machine learning" or "artificial intelligence" about Arts, History, Philosophy, Religion limited to document type(s) article, chapter, book from 2011 - 2020. There are 12,286 documents in this dataset in total. I have built this dataset before class. 

In [ ]:
# Import constellate
import constellate

In [ ]:
# Creating a variable `dataset_id` to hold our dataset ID
dataset_id = 'd6232206-93bf-f6b8-9ad2-b2add01cf231'

In [ ]:
# use constellate.download() to download the dataset
# in the Constellate Document Format (jsonl) and give the file a name
dataset_file = constellate.download(dataset_id, 'jsonl', 'ML_AI')

See the Constellate Client documentation at: https://constellate.org/docs/constellate-client for the different downloading options. 

## Read in the data
After we download the dataset, we can use the `dataset_reader()` method to read in the data. 

In [ ]:
# Use the .dataset_reader() method to read in the documents
docs = constellate.dataset_reader(dataset_file)

In [ ]:
# Check the type of docs
type(docs)

Recall from [Python Intermediate 5](./python-intermediate-5.ipynb) that the difference between a list and a generator is that the latter yields only one element at a time. As a result, generators are more memory-efficient than lists. 

To return the elements in a generator one by one, we use the `next()` function.

In [ ]:
### Take a look at the first element of the generator docs
doc1 = next(docs)
doc1

We can see that the document is loaded as a Python dictionary. 

In [ ]:
# Get all keys from the dict
doc1.keys()

<h2 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h2>

Now you know that a Constellate dataset of documents with metadata, ngrams and possibly full text is read into a generator of dictionaries. Can you follow the prompts below to create a dataframe out of a Constellate dataset? Use what you have learned from [Python Basics](./python-basics-1.ipynb), [Pandas 1](./pandas-1.ipynb) and [Pandas 2](./pandas-2.ipynb) to do this exercise.

Step 1. Complete the following code cell to download a Constellate dataset.

In [ ]:
# Read in the data from a Constellate dataset
dataset_id = 'f6ae29d4-3a70-36ee-d601-20a8c0311273'

# Use constellate.get_dataset() to download the dataset(sampled to 1500 documents)
path = 

Step 2. In the next code cell, read in the data from the dataset 

In [ ]:
# Use constellate.dataset_reader() to read in the data
docs = 

Step 3. Get the keys from the first document in the dataset

In [ ]:
# Grab the first document from the dataset
doc1 = next(data)

# Get the keys from the first doc1


In [ ]:
# Select the keys that are of interest to you and
# put them in a list called keys_of_interest
keys_of_interest = 

Step 4. In the next code cell, create a dataframe storing the data of interest to you. The headers of the dataframe are the keys of interest you have just selected. Each row of the dataframe contains the relevant data from one document of the dataset. For example, if you choose 'id' and 'publicationYear' as the keys of interest. Then, the first row of the dataframe will have the id of the first document in the 'id' column and the publication year of the first document in the 'publicationYear' column. 

In [ ]:
# import pandas
import pandas as pd

# Get the docs again
docs = constellate.dataset_reader(path)

# Create a dataframe 


## Create a dataframe

In [ ]:
# import the Pandas library
import pandas as pd

Suppose not all data in the documents are of interest to us. Let's select the data we are interested in.  

In [ ]:
# Data of interest
data_of_interest = ['id', 'title', 'docType', 'publicationYear', 'bigramCount']

In [ ]:
# Get the docs again
docs = constellate.dataset_reader(dataset_file)

From each doc in docs, we want to grab the values corresponding to the keys in the list of `data_of_interest` and create a dataframe from the data. For a quick review of list comprehensions, take a look at [Python Intermediate 1](./python-intermediate-1.ipynb).

In [ ]:
# Get all the data we need for creating a dataframe
data = [[doc['id'], doc['title'], doc['docType'], doc['publicationYear'], doc['bigramCount']] for doc in docs]

In [ ]:
# Create a dataframe
df = pd.DataFrame(data, columns=['id', 'title', 'docType', 'publicationYear', 'bigramCount'])
df

## Data cleaning and pre-processing

We will often need to do some data cleaning and pre-processing after we create a dataframe. What kind of data cleaning and pre-processing you need to do depends on the specific task at hand. Here, I only give some examples. 

When we look at the 'id' column, we can see that all document ids start with "ark://27972/". We can get rid of this prefix and use the rest of the string as the ids. 

In [ ]:
# Shorten the ids
df['id'] = df['id'].apply(lambda r: r.rsplit('/')[-1])
df

<h2 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h2>

The bigramCount column gives the number of occurrences of every bigram string in a document. As you can see, the puntuations do not count as a gram. This is why 'life. But' is seen as a bigram, not a trigram. With this in mind, can you make a new column storing the count of the bigram 'machine learning' and a new column storing the bigram 'artificial in telligence'? You can drop the bigramCount column after making the two columns. 

In [ ]:
# Example solution 
len_ML = len('machine learning')

# Define a function to return the count of 'machine learning' and 'artificial intelligence'
def count(r):
    count_ML = 0
    count_AI = 0
    for key in r.keys():
        key_lower = key.lower()
        if len(key)<len_ML:
            continue
        elif 'machine learning' in key_lower:
            count_ML += r[key]
        elif 'artificial intelligence' in key_lower:
            count_AI += r[key]
    return [count_ML, count_AI]

# Create a column storing the count of 'machine learning'
df[['ML_count', 'AI_count']] = df['bigramCount'].apply(count).tolist()

In [ ]:
# Take a look at the updated df
df

## Group and aggregate data

After data cleaning, filtering and preprocessing, the next step is to summarize the data to extract useful information.

Pandas makes summarising a dataframe very easy. For example, we can count how many non-null values there are in each column using the `.count()` method. 

In [ ]:
# Get the number of non-null values in each column
df.count()

We can also get the max value or the min value of a column using the `.max()` and `.min()` methods. 

In [ ]:
# Get the max value from the year column
df['publicationYear'].max()

In [ ]:
# Get the min value from the year column
df['publicationYear'].min()

You can refer to the Pandas documentation for more methods that you can use to query the data. 

When you summarize a dataframe, a very useful method is `.describe()`. It can quickly display the statistics for any group of data it is applied to. 

In [ ]:
# Use the .describe() method to explore the year column
df['title'].describe()

### Groupby()

Groupby is a powerful function built into Pandas that you can use to summarize your data. Groupby splits the data into different groups on a variable of your choice. 

In [ ]:
# Group the data by docType
df.groupby('docType')

The groupby() method returns a GroupBy object which describes how the rows of the original dataset have been split by the selected variable. You can actually see how the rows of the original dataframe have been grouped using `groups` after applying `groupby()`.

In [ ]:
# See how the rows have been grouped
df.groupby('docType').groups

As you can see, a dictionary is returned whose keys are the unique values in docType and whose values are lists of row indexes. Each key corresponds to a list of row indexes.

In [ ]:
# Get the keys from the groups dictionary
df.groupby('docType').groups.keys()

You can group the data using multiple variables. For example, you may want to group the documents first by their publication year and then by the document type. 

In [ ]:
# Group by a composite variable ['publicationYear', 'docType']
df.groupby(['publicationYear', 'docType'])

If you take a look at the groups in the groupby object, you will see that essentially we have a composite key for each group. The first key, for example, is (2011, 'article'). The value associated with this key is a list of indexes, all of which are the rows storing the documents that were published in 2011 and are of the docType 'article'.

In [ ]:
# Group by multiple variables 
# Take a look at the composite keys
df.groupby(['publicationYear', 'docType']).groups

Of course, we don't just stop at grouping data. Grouping data is just a step towards data query. After we apply the `.groupby()` method, we can actually use different Pandas methods to query the data. For example, how do we get the number of documents in each docType by publicationYear?

In [ ]:
# Create a series storing the number of documents in each doc type by year
df.groupby(['publicationYear', 'docType']).size()

### Agg() 

After we group the data in a dataframe, we can apply the `agg()` method to calculate multiple statistics per group in one calculation. 

For example, let's say we would like to know the sum of the occurrences of the bigram 'machine learning' in all the documents from each year. To achieve this goal, we can group the data by `publicationYear`, and then aggregate the data by summing the numerical values in the column of `ML_count` for each subgroup.  

In [ ]:
# Get how many times 'machine learning' is
# mentioned in the docs each year
df.groupby('publicationYear').agg({'ML_count':'sum'})

Of course, you can choose other ways to aggregate the data in each subgroup. For example, suppose you are interested in the biggest frequency with which a document mentions 'artificial intelligence' by year.

In [ ]:
# the biggest frequency of a document mentioning 
# 'artificial intelligence' by year
df.groupby('publicationYear').agg({'AI_count':'max'})

We can specify multiple columns to apply a function to. 

In [ ]:
# apply a single function to selected columns in each subgroup
df.groupby('publicationYear').agg({'AI_count':'sum', 'ML_count': 'max'})

We can also apply multiple functions to each of the selected columns.

In [ ]:
# apply multiple functions to selected columns in each subgroup
df.groupby('publicationYear').agg({'AI_count':['sum', 'max'], 'ML_count':['max', 'count']})

<h2 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h2>

Take the following dataframe containing the information on the Covid19 cases in the state of Massachusetts. Can you work with the dataframe to find out which month of which year has the most positive new cases?

In [ ]:
# Get the data of covid19 cases in MA and 
# create a dataframe 
import urllib
url = 'https://www.mass.gov/doc/covid-19-raw-data-march-9-2023/download'
urllib.request.urlretrieve(url, './data/covid_MA')
covid_ma = pd.read_excel(url, 'Cases (Report Date)')
covid_ma['Date'] = covid_ma['Date'].astype(str)

# Take a look at the dataframe
covid_ma

## Make pivot tables in Pandas

Pandas has a `.pivot_table()` method we can use to summarize data. It takes a dataframe as argument and has parameters specifying the shape of the pivot table. 

In the previous section, we have used the `.groupby()` and `agg()` methods to summarize data. For example, we grouped the documents in the dataframe df by their year of publication and calculated the sum of the mentions of the bigram 'artificial intelligence'. We can do the same thing using the `.pivot_table` method. 

In [ ]:
# Create a pivot table giving the sum of 
# the mentions of 'artificial intelligence' by year
df.pivot_table(index='publicationYear', 
                       values='AI_count',
                      aggfunc='sum')

Again, when aggregating the data, you can apply a single function to multiple columns. 

In [ ]:
# Create a pivot table giving the sum of 
# the mentions of 'machine learning' and 'artificial intelligence' by year
df.pivot_table(index='publicationYear', 
                       values=['AI_count', 'ML_count'],
                      aggfunc='sum')

You can also apply multiple functions to a single column. 

In [ ]:
# Create a pivot table giving the sum and the max value of
# the mentions of 'artificial intelligence' by year
df.pivot_table(index='publicationYear', 
                       values='AI_count',
                      aggfunc=['sum', 'max'])

Or, you can apply different functions to different columns. 

In [ ]:
# Create a pivot table giving the sum of
# the mentions of 'artificial intelligence' by year
# and the max value of the mentions of 'machine learning' by year
df.pivot_table(index='publicationYear', 
                       values=['AI_count', 'ML_count'],
                      aggfunc={'AI_count':'sum', 'ML_count':'max'})

<h2 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h2>

Get the dataframe stored in the variable `covid_ma`. Can you make a pivot table showing the sum of the positive cases from 2020 - 2023 in that table? 

## A teaser for the Data Visualization class

We have learned how to create a dataset from Constellate, how to preprocess the data and how to summarize the data. With the information we get from summarizing the data, we can go ahead and plot it!

For example, let's plot the number of docs that mentioned 'artificial intelligence' or 'machine learning' from 2011 - 2020. 

In [ ]:
# Prepare the dataframe for plotting
df.groupby('publicationYear').size().plot(kind='bar', ylabel='num_doc')

___
# Lesson Complete
Congratulations! You've completed the *Pandas* series. 

Considering the amount of material in *Pandas 1-3* there's a good chance you won't retain it all. That's okay. Programmers often need to look up things to accomplish a task they haven't done in a while, particularly if it is in a language they don't often use. When you're working on a project, you can always come back to these lessons as reference materials. In other words, you've learned an incredible amount, so don't be surprised if it doesn't all stick at first.

If you want to help yourself retain what you've learned, the best way is to start putting it into practice. Try your hand at creating some small Pandas projects and recognize that the things you've learned here will cement with time and practice. When you do forget a particular thing&mdash;as we all do&mdash;a quick web search often turns up some useful examples.

## Start a Text Analysis Lesson:
* [Exploring Metadata](./exploring-metadata.ipynb) 

## Solutions to exercises

Here are the solutions to some of the exercises in this notebook.

In [ ]:
### Find out which month of which year has the most positive cases of covid19 in MA

# Download the file
import urllib
url = 'https://www.mass.gov/doc/covid-19-raw-data-march-9-2023/download'
urllib.request.urlretrieve(url, './data/covid_MA')

# Create a dataframe from the relevant sheet
covid_ma = pd.read_excel(url, 'Cases (Report Date)')
covid_ma['Date'] = covid_ma['Date'].astype(str) # process the date column

# Extract the year and month from the date column
covid_ma['group_var'] = covid_ma['Date'].apply(lambda r: r.rsplit('-', 1)[0])

# Use the new column as a grouping variable and divide the data into subgroups
# Aggregate the data using 'sum' function and sort the results in descending order
covid_ma.groupby('group_var').agg({'Positive New':'sum'}).sort_values('Positive New', ascending=0)

In [ ]:
### Make a pivot table showing the sum of positive covid19 cases by year in covid_ma

covid_ma['Year'] = covid_ma['Date'].str.slice(0,4)
covid_ma.pivot_table(index='Year',
                    values='Positive New',
                    aggfunc='sum')